In [ ]:
# H

# Tools

## Data preprocessing

Load a month of monthly Neighborhood Patterns, then prepare data for IPF.

In [27]:
%load_ext autoreload
%autoreload 2
import os
import random
import sqlite3 
import numpy as np
import json
import math

import pandas as pd

import Advan_operator as ad_op  

pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load monthly Neighborhood Patterns

#### Asign file locations

In [145]:
# data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Lenovo


data_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 
save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # Dell

year = '2023'
month = 6

ACS_file = r"F:\Research\Wild_fire\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
home_panel_fname = f"F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv"


month_dir = os.path.join(data_dir, str(year), f'{month:02}', '01')
no_dict_colum_fname = os.path.join(save_dir, f"NP_dict_column_{year}_{month:02}.csv")
device_home_areas_fname = os.path.join(save_dir, f"NP_split_device_home_areas_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_CBG_fname     = os.path.join(save_dir, f"adjusted_dwelling_time_stop_CBG_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_natinol_fname = os.path.join(save_dir, f"adjusted_dwelling_time_stop_natinol_{year}_{month:02}.csv")
person_per_device_fname = os.path.join(save_dir, f"person_per_device_{year}_{month:02}.csv")
restored_CBG_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_CBG_{year}_{month:02}.csv")
restored_target_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_target_{year}_{month:02}.csv")


# month_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01'
# CBG_list_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\ACS_2019_CBG_list.csv"

sqlite_fname = r'F:\Research\Wild_fire\Neighborhood_202306.db'


print("Month directory:", month_dir)
print("Save_dir directory:", save_dir)
print("ACS_file:", ACS_file)
print("home_panel file name:", home_panel_fname)

print("No dict colum file name:", no_dict_colum_fname)
print("Device home areas file name:", device_home_areas_fname)
print("Device person_per_device file name:", person_per_device_fname)


print("Device adjusted_dwelling_time_stop_CBG_fname:", adjusted_dwelling_time_stop_CBG_fname)
print("Device adjusted_dwelling_time_stop_natinol_fname:", adjusted_dwelling_time_stop_natinol_fname)

print("Restored_hourly_vistor_count_CBG file name:   ", restored_CBG_hourly_vistor_count_fname)
print("Restored_hourly_vistor_count_target file name:", restored_target_hourly_vistor_count_fname)

Month directory: K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01
Save_dir directory: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test
ACS_file: F:\Research\Wild_fire\cbg_acs_2019_county_tract_new20230929_cleaned.csv
home_panel file name: F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv
No dict colum file name: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\NP_dict_column_2023_06.csv
Device home areas file name: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\NP_split_device_home_areas_2023_06.csv
Device person_per_device file name: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\person_per_device_2023_06.csv
Device adjusted_dwelling_time_stop_CBG_fname: D:\OneDrive_PSU\OneDrive - The Pennsylvania State Un

In [10]:
# Load monthly neighborhood patterns

month_df = ad_op.load_neighborhood_monthly_folder(folder=month_dir, extions=['gz'], verbose=True)
month_df

Found target files: 22
The top 5 and bottom 5 files:
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\data_01af1467-0604-be6e-0043-0b8700e85436_53_1_0.csv.gz
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\data_01af1467-0604-be6e-0043-0b8700e85436_53_1_1.csv.gz
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\data_01af1467-0604-be6e-0043-0b8700e85436_53_1_10.csv.gz
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\data_01af1467-0604-be6e-0043-0b8700e85436_53_1_2.csv.gz
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\data_01af1467-0604-be6e-0043-0b8700e85436_53_1_3.csv.gz
...
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\Neighborhood_Patterns_US-658-DATE_RANGE_START-2023-06-01.csv.gz
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\Neighborhood_Patterns_US-659-DATE_RANGE_START-2023-06-01.csv.gz
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\Neighborhood_Patterns_US-660-DATE_RANGE_START-

AREA           AREA_TYPE    ORIGIN_AREA_TYPE  \
0      470370112006  Census Block Group  Census Block Group   
1      310199694003  Census Block Group  Census Block Group   
2      550939606003  Census Block Group  Census Block Group   
3      450770112042  Census Block Group  Census Block Group   
4       60375545222  Census Block Group  Census Block Group   
...             ...                 ...                 ...   
20952   60375433211  Census Block Group  Census Block Group   
20953  390375701021  Census Block Group  Census Block Group   
20954  201259509002  Census Block Group  Census Block Group   
20955  550790190003  Census Block Group  Census Block Group   
20956  540879631003  Census Block Group  Census Block Group   

              DATE_RANGE_START           DATE_RANGE_END  \
0      2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
1      2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
2      2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
3      2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
4      2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
...                        ...                      ...   
20952  2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
20953  2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
20954  2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
20955  2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   
20956  2023-06-01 00:00:00.000  2023-07-01 00:00:00.000   

                                              DAY_COUNTS  RAW_STOP_COUNTS  \
0      {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...             6558   
1      {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...            42090   
2      {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...            33042   
3      {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...            11249   
4      {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...            14560   
...                                                  ...              ...   
20952  {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...            76304   
20953  {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...            12605   
20954  {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...            26633   
20955  {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...             4628   
20956  {"Monday":4,"Tuesday":4,"Wednesday":4,"Thursda...             5960   

       RAW_DEVICE_COUNTS                                       STOPS_BY_DAY  \
0                   4181  [208,162,223,224,92,215,261,174,260,218,326,15...   
1                   8019  [1842,1829,1559,1210,1200,1150,1338,1712,1755,...   
2                   4781  [1594,1195,789,1047,930,927,726,928,1295,1130,...   
3                   3083  [280,354,256,461,475,677,451,321,401,479,187,2...   
4                   6277  [313,430,408,450,351,886,451,358,432,808,613,2...   
...                  ...                                                ...   
20952              13849  [2619,2600,2303,1380,3382,2773,2634,2639,2966,...   
20953               1413  [241,129,294,223,215,303,204,457,451,513,344,3...   
20954               4948  [958,1126,1618,804,807,995,980,1092,1460,705,5...   
20955               1532  [142,92,79,77,133,149,173,175,280,127,78,132,1...   
20956                931  [64,144,188,154,112,162,168,199,141,171,69,143...   

                                      STOPS_BY_EACH_HOUR  \
0      [0,0,0,0,0,0,0,0,22,22,39,6,14,26,28,0,7,10,5,...   
1      [0,10,17,23,51,16,89,56,90,133,116,174,183,106...   
2      [0,0,5,42,74,42,23,59,82,145,70,72,81,71,64,11...   
3      [42,0,0,0,0,0,10,28,16,0,17,14,8,36,11,14,0,51...   
4      [0,0,0,0,0,14,0,0,0,24,0,36,22,22,0,17,37,0,96...   
...                                                  ...   
20952  [0,39,38,24,0,43,60,329,241,138,211,172,158,21...   
20953  [0,0,0,0,0,0,0,0,15,26,9,0,7,13,39,6,14,0,23,1...   
20954  [5,0,0,0,8,8,39,50,85,55,36,28,59,132,41,82,61...   
20955  [15,0,0,0,0,0,0,0,0,10,0,14,0,0,8,8,17,0,25,31...   
20956  [11,0,0,

### Save the a single CSV for further use, only keep the non-dictionary columns.

In [33]:
non_dict_columns = ['AREA', 
                    "STOPS_BY_DAY", 
                    "RAW_STOP_COUNTS", 
                    "RAW_DEVICE_COUNTS", 
                    "MEDIAN_DWELL", 
                    "STOPS_BY_EACH_HOUR"]
# save CSV file
# month_df[non_dict_columns].to_csv(no_dict_colum_fname, index=False)
# print("Combined CSV wihouth dictionary columns was saved at:", no_dict_colum_fname)

In [162]:
### Split the DEVICE_HOME_AREAS column

In [61]:
month_df = month_df.sort_values("AREA")
device_home_areas_df = ad_op.split_a_dictionary_column(np_df=month_df.iloc[:], dest_col='AREA', origin_col='DEVICE_HOME_AREAS', value_name='device')
print("Sorting...")
# device_home_areas_df = device_home_areas_df.sort_values(['origin', 'destination'])
# device_home_areas_df = device_home_areas_df.sort_values(['origin', 'device'])
device_home_areas_df['origin'] = device_home_areas_df['origin'].astype(str).str.zfill(12)
device_home_areas_df['destination'] = device_home_areas_df['destination'].astype(str).str.zfill(12)
device_home_areas_df['device'] = device_home_areas_df['device'].astype(int)

print("Saving...")
device_home_areas_df.to_csv(device_home_areas_fname, index=False)
device_home_areas_df

Sorting...
Saving...


origin   destination  device
0         010010201001  010010201001      84
1         010010208021  010010201001      63
2         010510313001  010010201001      53
3         010010205001  010010201001      42
4         011010059011  010010201001      32
...                ...           ...     ...
99501279  370810105002  780309900000       4
99501280  530330284024  780309900000       4
99501281  780309601002  780309900000       4
99501282  120860090361  780309900000       4
99501283  483396920021  780309900000       4

[99501284 rows x 3 columns]

###	Adjust stop count by dwelling time

Load the raw stop data and Neighorhood Patterns table.

In [3]:
# Load the raw stop data and Neighorhood Patterns table.

# device_home_areas_df = pd.read_csv(device_home_areas_fname)
print("Loading no_dict_colum CSV file:", no_dict_colum_fname)
no_dict_colum_df = pd.read_csv(no_dict_colum_fname)

no_dict_colum_df

AREA                                       STOPS_BY_DAY  \
0       470370112006  [208,162,223,224,92,215,261,174,260,218,326,15...   
1       310199694003  [1842,1829,1559,1210,1200,1150,1338,1712,1755,...   
2       550939606003  [1594,1195,789,1047,930,927,726,928,1295,1130,...   
3       450770112042  [280,354,256,461,475,677,451,321,401,479,187,2...   
4        60375545222  [313,430,408,450,351,886,451,358,432,808,613,2...   
...              ...                                                ...   
220679   60375433211  [2619,2600,2303,1380,3382,2773,2634,2639,2966,...   
220680  390375701021  [241,129,294,223,215,303,204,457,451,513,344,3...   
220681  201259509002  [958,1126,1618,804,807,995,980,1092,1460,705,5...   
220682  550790190003  [142,92,79,77,133,149,173,175,280,127,78,132,1...   
220683  540879631003  [64,144,188,154,112,162,168,199,141,171,69,143...   

        RAW_STOP_COUNTS  RAW_DEVICE_COUNTS  MEDIAN_DWELL  \
0                  6558               4181     32.066667   
1                 42090               8019     10.483333   
2                 33042               4781     11.366667   
3                 11249               3083     22.566667   
4                 14560               6277     25.483333   
...                 ...                ...           ...   
220679            76304              13849     55.333333   
220680            12605               1413     20.883333   
220681            26633               4948     20.033333   
220682             4628               1532     40.950000   
220683             5960                931     10.033333   

                                       STOPS_BY_EACH_HOUR  
0       [0,0,0,0,0,0,0,0,22,22,39,6,14,26,28,0,7,10,5,...  
1       [0,10,17,23,51,16,89,56,90,133,116,174,183,106...  
2       [0,0,5,42,74,42,23,59,82,145,70,72,81,71,64,11...  
3       [42,0,0,0,0,0,10,28,16,0,17,14,8,36,11,14,0,51...  
4       [0,0,0,0,0,14,0,0,0,24,0,36,22,22,0,17,37,0,96...  
...                                                   ...  
220679  [0,39,38,24,0,43,60,329,241,138,211,172,158,21...  
220680  [0,0,0,0,0,0,0,0,15,26,9,0,7,13,39,6,14,0,23,1...  
220681  [5,0,0,0,8,8,39,50,85,55,36,28,59,132,41,82,61...  
220682  [15,0,0,0,0,0,0,0,0,10,0,14,0,0,8,8,17,0,25,31...  
220683  [11,0,0,0,0,0,0,0,13,23,0,0,0,17,0,0,0,0,9,0,0...  

[220684 rows x 6 columns]

In [4]:
non_adjusted_dwell_time_stop_CBG_arr = ad_op.adjust_stop_by_dwelling_time(np_df=no_dict_colum_df.iloc[:], adjust_dwell_time=False, clean_negative=True)
print("sum of non_adjusted_dwell_time_stop_CBG_arr (million):", non_adjusted_dwell_time_stop_CBG_arr.sum().sum()/1000000)
print("shape of non_adjusted_dwell_time_stop_CBG_arr:", non_adjusted_dwell_time_stop_CBG_arr.shape)

sum of non_adjusted_dwell_time_stop_CBG_arr (million): 4099.559846
shape of non_adjusted_dwell_time_stop_CBG_arr: (220684, 720)


In [5]:
adjusted_dwell_time_stop_CBG_arr = ad_op.adjust_stop_by_dwelling_time(np_df=no_dict_colum_df.iloc[:], adjust_dwell_time=True, clean_negative=True)

adjusted_dwell_time_stop_national_arr = adjusted_dwell_time_stop_CBG_arr.sum(axis=0)

print("sum of adjusted_dwell_time_stop_CBG_arr (million):", adjusted_dwell_time_stop_CBG_arr.sum().sum()/1000000)
print("sum of adjusted_dwell_time_stop_national_arr (million):", adjusted_dwell_time_stop_national_arr.sum().sum()/1000000)
print("shape of adjusted_dwell_time_stop_CBG_arr:", adjusted_dwell_time_stop_CBG_arr.shape)

sum of adjusted_dwell_time_stop_CBG_arr (million): 5196.196864
sum of adjusted_dwell_time_stop_national_arr (million): 5196.196864
shape of adjusted_dwell_time_stop_CBG_arr: (220684, 720)


In [64]:
# hourly_columns
# adjusted_dwell_time_stop_national_arr.shape

(720,)

Convert numpy array to DataFrame

In [11]:
# convert numpy array to DataFrame
# CBG level
hourly_columns = [f'hour_{h+1}' for h in range(adjusted_dwell_time_stop_CBG_arr.shape[1])]
adjusted_dwelling_time_stop_CBG_df = pd.DataFrame(adjusted_dwell_time_stop_CBG_arr, columns=hourly_columns)
adjusted_dwelling_time_stop_CBG_df['CBG'] = no_dict_colum_df['AREA'].astype(str).str.zfill(12).to_list()
adjusted_dwelling_time_stop_CBG_df = adjusted_dwelling_time_stop_CBG_df[['CBG'] + hourly_columns]  # reorder columns
adjusted_dwelling_time_stop_CBG_df = adjusted_dwelling_time_stop_CBG_df.sort_values('CBG')


# national level
adjusted_dwelling_time_stop_natinol_df = pd.DataFrame(adjusted_dwell_time_stop_national_arr, columns=['stop'])

adjusted_dwelling_time_stop_CBG_df

CBG  hour_1  hour_2  hour_3  hour_4  hour_5  hour_6  hour_7  \
141400  010010201001       0       0      23       0       0       0       0   
38756   010010201002      33      33      33       0       0       0       0   
169554  010010202001      14      24       0       0       0       0       0   
117119  010010202002      92      20       0       0       0      14       0   
141640  010010203001      33       0       0       0       0       0       0   
...              ...     ...     ...     ...     ...     ...     ...     ...   
39198   780309611001       2       0       0       7       4       4      11   
189527  780309611002       0       0       0       0       0       0       9   
210907  780309612001       2       0       0       0       0       0       6   
67908   780309612002       2       0       0       0       6       5       2   
55867   780309900000      10       2       4       7       7       0       2   

        hour_8  hour_9  hour_10  hour_11  hour_12  hour_13  hour_14  hour_15  \
141400       0       0        0        0       43       23       14       19   
38756        0       0        0       22       43       43       21       21   
169554       0       0       16        0       19       16        0       18   
117119     153      95       42       61       42       21       52       60   
141640      24      41       17       58       97       79       40       62   
...        ...     ...      ...      ...      ...      ...      ...      ...   
39198       10      16       26       15       16       44       20       17   
189527      16      22       25       13       12       23       18       22   
210907       6       3        5        5        3        2        3        5   
67908       26      11       26       16       10       20       14       10   
55867        0       0        0        0        4        0        0        0   

        hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  \
141400        0       15       52       22       14       20       19   
38756        21       41       63       83       84       65       65   
169554        0       22       37        0        0       23        0   
117119       38       72       75      132       33        0       14   
141640       80       58       64       76       52       60       80   
...         ...      ...      ...      ...      ...      ...      ...   
39198        18       22       33       23        2        0        1   
189527       12       21       21        5        4       12        8   
210907       10        6       10        4        3        0        5   
67908        21       18       16       18       16        0        1   
55867         3        8        2        0        3        0        0   

        hour_23  hour_24  hour_25  hour_26  hour_27  hour_28  hour_29  \
141400       22        0        0       19        0       16        0   
38756        63       39       19        0        0        0        0   
169554       22       24        0        0        0        0        0   
117119       42       22       22        0        0        0        0   
141640       81       85       48       24        0        0        0   
...         ...      ...      ...      ...      ...      ...      ...   
39198         8        0        0        0        0        0        0   
189527        4        2        0        5        0        0        0   
210907        0        0        0        0        0        0        0   
67908         0       10        9        5        0        1        5   
55867         0        0        0        0        0        0        0   

        hour_30  hour_31  hour_32  hour_33  hour_34  hour_35  hour_36  \
141400        0       24       23        0        0       18        0   
38756         0        0       16       38      114      137      115   
169554        0        0       23        0        0        0        0   
117119        0        0       78       73       

In [19]:
print("Adjustted total stop by dwelling time from CBG and national level (should be the same:")
print(adjusted_dwelling_time_stop_CBG_df[hourly_columns].sum().sum(),  adjusted_dwelling_time_stop_natinol_df.sum().sum())

Adjustted total stop by dwelling time from CBG and national level (should be the same:
5196196864 5196196864


In [20]:
# Save CSV files
adjusted_dwelling_time_stop_natinol_df.to_csv(adjusted_dwelling_time_stop_natinol_fname, index=False)
print("Saved national hourly stop at:", adjusted_dwelling_time_stop_natinol_fname)

adjusted_dwelling_time_stop_CBG_df.to_csv(adjusted_dwelling_time_stop_CBG_fname, index=False)
print("Saved CBG hourly stop at:", adjusted_dwelling_time_stop_CBG_fname)

Saved national hourly stop at: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\adjusted_dwelling_time_stop_natinol_2023_06.csv
Saved CBG hourly stop at: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\adjusted_dwelling_time_stop_CBG_f_2023_06.csv


###  Convert observed stop counts to actual visitor counts.

#### compute the person_per_device at CBG and county level

Load ACS and home panel data, merge them into device sampling rate table at CBG and county level

In [33]:
ACS_df = pd.read_csv(ACS_file)
ACS_df["CBG_FIPS"] = ACS_df["fips"].astype(str).str.zfill(12)
print("Total US population (million):",  ACS_df['totalpopulation'].sum() / 1000000)

Total US population (million): 328.016242


In [38]:
print("Loading home panel file:", home_panel_fname)
panel_df = pd.read_csv(home_panel_fname)
target_panel_df = panel_df.query(f"YEAR == {year} and MON == {int(month)} and ISO_COUNTRY_CODE == 'US' ")
target_panel_df

Loading home panel file: F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv


YEAR  MON REGION ISO_COUNTRY_CODE CENSUS_BLOCK_GROUP  \
547544  2023    6     AK               US       020200013002   
547545  2023    6     AK               US       021880001005   
547546  2023    6     AK               US       020900011001   
547547  2023    6     AK               US       021880002003   
547548  2023    6     AK               US       021100002002   
...      ...  ...    ...              ...                ...   
812375  2023    6     WY               US       560050004001   
812376  2023    6     WY               US       560250005012   
812377  2023    6     WY               US       560319594001   
812378  2023    6     WY               US       560099566002   
812379  2023    6     WY               US       560099566001   

        NUMBER_DEVICES_RESIDING  NUMBER_DEVICES_PRIMARY_DAYTIME  
547544                     18.0                             NaN  
547545                      6.0                             NaN  
547546                    664.0                           768.0  
547547                      6.0                             6.0  
547548                     18.0                             6.0  
...                         ...                             ...  
812375                    268.0                           250.0  
812376                    286.0                           250.0  
812377                    195.0                           250.0  
812378                    225.0                           250.0  
812379                    524.0                           506.0  

[220039 rows x 7 columns]

In [49]:
panel_ACS_df = target_panel_df.merge(ACS_df[['CBG_FIPS', 'totalpopulation']], left_on='CENSUS_BLOCK_GROUP', right_on='CBG_FIPS')
panel_ACS_df = panel_ACS_df.dropna(subset='NUMBER_DEVICES_RESIDING')
panel_ACS_df['NUMBER_DEVICES_RESIDING'] = panel_ACS_df['NUMBER_DEVICES_RESIDING'].astype(int)
panel_ACS_df = panel_ACS_df.sort_values('CBG_FIPS')
panel_ACS_df['person_per_device'] = panel_ACS_df['totalpopulation'] / panel_ACS_df['NUMBER_DEVICES_RESIDING']
print("Data cover US population:", panel_ACS_df['totalpopulation'].sum())
panel_ACS_df

Data cover US population: 327227353


YEAR  MON REGION ISO_COUNTRY_CODE CENSUS_BLOCK_GROUP  \
1248    2023    6     AL               US       010010201001   
1991    2023    6     AL               US       010010201002   
1152    2023    6     AL               US       010010202001   
2905    2023    6     AL               US       010010202002   
934     2023    6     AL               US       010010203001   
...      ...  ...    ...              ...                ...   
175266  2023    6     PR               US       721537506011   
176252  2023    6     PR               US       721537506012   
175558  2023    6     PR               US       721537506013   
175994  2023    6     PR               US       721537506021   
174889  2023    6     PR               US       721537506022   

        NUMBER_DEVICES_RESIDING  NUMBER_DEVICES_PRIMARY_DAYTIME      CBG_FIPS  \
1248                         61                            54.0  010010201001   
1991                        134                            97.0  010010201002   
1152                         42                            48.0  010010202001   
2905                         85                           158.0  010010202002   
934                         274                           286.0  010010203001   
...                         ...                             ...           ...   
175266                       73                            85.0  721537506011   
176252                      244                           189.0  721537506012   
175558                      134                           103.0  721537506013   
175994                      122                           146.0  721537506021   
174889                       54                            67.0  721537506022   

        totalpopulation  person_per_device  
1248                730          11.967213  
1991               1263           9.425373  
1152                835          19.880952  
2905               1124          13.223529  
934                2774          10.124088  
...                 ...                ...  
175266              883          12.095890  
176252             2523          10.340164  
175558              991           7.395522  
175994             1577          12.926230  
174889              648          12.000000  

[219124 rows x 10 columns]

In [81]:
# get people_per_device of a county

panel_ACS_df['county_FIPS'] = panel_ACS_df['CBG_FIPS'].str[:5]
sampling_rate_county_df = panel_ACS_df.groupby(['county_FIPS'], as_index=False).agg(    # , 'YEAR', "MON", "REGION", "ISO_COUNTRY_CODE"
    county_device=('NUMBER_DEVICES_RESIDING', 'sum'),
    county_device_daytime=('NUMBER_DEVICES_RESIDING', 'sum'),
    county_device_population=('totalpopulation', 'sum'),
    ) 

sampling_rate_county_df['person_per_device_county'] = sampling_rate_county_df['county_device_population'] /  sampling_rate_county_df['county_device']

print("National device and population count:\n")
print(sampling_rate_county_df[['county_device', 'county_device_population']].sum())

people_per_device_df = panel_ACS_df.merge(sampling_rate_county_df, left_on='county_FIPS', right_on='county_FIPS')
people_per_device_df.to_csv(person_per_device_fname, index=False)
people_per_device_df

National device and population count:

county_device                45282809
county_device_population    327227353
dtype: int64


YEAR  MON REGION ISO_COUNTRY_CODE CENSUS_BLOCK_GROUP  \
0       2023    6     AL               US       010010201001   
1       2023    6     AL               US       010010201002   
2       2023    6     AL               US       010010202001   
3       2023    6     AL               US       010010202002   
4       2023    6     AL               US       010010203001   
...      ...  ...    ...              ...                ...   
219119  2023    6     PR               US       721537506011   
219120  2023    6     PR               US       721537506012   
219121  2023    6     PR               US       721537506013   
219122  2023    6     PR               US       721537506021   
219123  2023    6     PR               US       721537506022   

        NUMBER_DEVICES_RESIDING  NUMBER_DEVICES_PRIMARY_DAYTIME      CBG_FIPS  \
0                            61                            54.0  010010201001   
1                           134                            97.0  010010201002   
2                            42                            48.0  010010202001   
3                            85                           158.0  010010202002   
4                           274                           286.0  010010203001   
...                         ...                             ...           ...   
219119                       73                            85.0  721537506011   
219120                      244                           189.0  721537506012   
219121                      134                           103.0  721537506013   
219122                      122                           146.0  721537506021   
219123                       54                            67.0  721537506022   

        totalpopulation  person_per_device county_FIPS  county_device  \
0                   730          11.967213       01001           8252   
1                  1263           9.425373       01001           8252   
2                   835          19.880952       01001           8252   
3                  1124          13.223529       01001           8252   
4                  2774          10.124088       01001           8252   
...                 ...                ...         ...            ...   
219119              883          12.095890       72153           4885   
219120             2523          10.340164       72153           4885   
219121              991           7.395522       72153           4885   
219122             1577          12.926230       72153           4885   
219123              648          12.000000       72153           4885   

        county_device_daytime  county_device_population  \
0                        8252                     55380   
1                        8252                     55380   
2                        8252                     55380   
3                        8252                     55380   
4                        8252                     55380   
...                       ...                       ...   
219119                   4885                     35428   
219120                   4885                     35428   
219121                   4885                     35428   
219122                   4885                     35428   
219123                   4885                     35428   

        person_per_device_county  
0                       6.711100  
1                       6.711100  
2                       6.711100  
3                       6.711100  
4                       6.711100  
...                          ...  
219119                  7.252405  
219120                  7.252405  
219121                  7.252405  
219122                  7.252405  
219123                  7.252405  

[219124 rows x 15 columns]

#### restore the observated stop count to actual visitor count

In [107]:
people_per_device_df = pd.read_csv(person_per_device_fname, dtype={'CBG_FIPS': str})
adjusted_dwelling_time_stop_CBG_df = pd.read_csv(adjusted_dwelling_time_stop_CBG_fname, dtype={'CBG': str})

In [76]:
adjusted_dwelling_time_stop_CBG_df
# people_per_device_df

CBG  hour_1  hour_2  hour_3  hour_4  hour_5  hour_6  hour_7  \
0       010010201001       0       0      23       0       0       0       0   
1       010010201002      33      33      33       0       0       0       0   
2       010010202001      14      24       0       0       0       0       0   
3       010010202002      92      20       0       0       0      14       0   
4       010010203001      33       0       0       0       0       0       0   
...              ...     ...     ...     ...     ...     ...     ...     ...   
220679  780309611001       2       0       0       7       4       4      11   
220680  780309611002       0       0       0       0       0       0       9   
220681  780309612001       2       0       0       0       0       0       6   
220682  780309612002       2       0       0       0       6       5       2   
220683  780309900000      10       2       4       7       7       0       2   

        hour_8  hour_9  hour_10  hour_11  hour_12  hour_13  hour_14  hour_15  \
0            0       0        0        0       43       23       14       19   
1            0       0        0       22       43       43       21       21   
2            0       0       16        0       19       16        0       18   
3          153      95       42       61       42       21       52       60   
4           24      41       17       58       97       79       40       62   
...        ...     ...      ...      ...      ...      ...      ...      ...   
220679      10      16       26       15       16       44       20       17   
220680      16      22       25       13       12       23       18       22   
220681       6       3        5        5        3        2        3        5   
220682      26      11       26       16       10       20       14       10   
220683       0       0        0        0        4        0        0        0   

        hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  \
0             0       15       52       22       14       20       19   
1            21       41       63       83       84       65       65   
2             0       22       37        0        0       23        0   
3            38       72       75      132       33        0       14   
4            80       58       64       76       52       60       80   
...         ...      ...      ...      ...      ...      ...      ...   
220679       18       22       33       23        2        0        1   
220680       12       21       21        5        4       12        8   
220681       10        6       10        4        3        0        5   
220682       21       18       16       18       16        0        1   
220683        3        8        2        0        3        0        0   

        hour_23  hour_24  hour_25  hour_26  hour_27  hour_28  hour_29  \
0            22        0        0       19        0       16        0   
1            63       39       19        0        0        0        0   
2            22       24        0        0        0        0        0   
3            42       22       22        0        0        0        0   
4            81       85       48       24        0        0        0   
...         ...      ...      ...      ...      ...      ...      ...   
220679        8        0        0        0        0        0        0   
220680        4        2        0        5        0        0        0   
220681        0        0        0        0        0        0        0   
220682        0       10        9        5        0        1        5   
220683        0        0        0        0        0        0        0   

        hour_30  hour_31  hour_32  hour_33  hour_34  hour_35  hour_36  \
0             0       24       23        0        0       18        0   
1             0        0       16       38      114      137      115   
2             0        0       23        0        0        0        0   
3             0        0       78       73       

In [78]:
adjusted_dwelling_time_stop_CBG_df.shape

(220684, 721)

In [135]:
# Assume a stop is a device
hour_cnt = adjusted_dwelling_time_stop_CBG_df.shape[1] - 1


hourly_stop_sampling_rate_df = adjusted_dwelling_time_stop_CBG_df.merge(people_per_device_df, left_on='CBG', right_on='CBG_FIPS')
national_people_per_device = hourly_stop_sampling_rate_df['totalpopulation'].sum() / hourly_stop_sampling_rate_df['NUMBER_DEVICES_RESIDING'].sum()

print("hour_cnt:", hour_cnt)
print("national_people_per_device:", national_people_per_device)

for h in range(hour_cnt):
    hourly_stop_sampling_rate_df.iloc[:, h + 1] = hourly_stop_sampling_rate_df.iloc[:, h + 1] * hourly_stop_sampling_rate_df['person_per_device']   # CBG leval
    # hourly_stop_sampling_rate_df.iloc[:, h + 1] = hourly_stop_sampling_rate_df.iloc[:, h + 1] * national_people_per_device    # national leval
    # hourly_stop_sampling_rate_df.iloc[:, h + 1] = hourly_stop_sampling_rate_df.iloc[:, h + 1] * hourly_stop_sampling_rate_df['person_per_device_county']   # county level
    

## used      CBG level sampling rate: 51028323628.24674
## used   county level sampling rate: 51223351774.32022   # increase 0.3%
## used national level sampling rate: 37518194037.32022   # decrease 26.5%

restored_actual_CBG_hourly_visitor_df = hourly_stop_sampling_rate_df.iloc[:, :hour_cnt + 1]

print("Restored total visitor count (billion):")
print(restored_actual_CBG_hourly_visitor_df.iloc[:, 1:hour_cnt + 1].sum().sum() / 1000000000)  # used CBG level sampling rate: 51028323628.24674

print("Restored total visitor dataframe shape:",  restored_actual_CBG_hourly_visitor_df.shape)

hour_cnt: 720
national_people_per_device: 7.226305160482226
Restored total visitor count (billion):
51.028323628246746
Restored total visitor dataframe shape: (219123, 721)


In [136]:
restored_actual_CBG_hourly_visitor_df

CBG       hour_1      hour_2      hour_3  hour_4     hour_5  \
0       010010201001     0.000000    0.000000  275.245902     0.0   0.000000   
1       010010201002   311.037313  311.037313  311.037313     0.0   0.000000   
2       010010202001   278.333333  477.142857    0.000000     0.0   0.000000   
3       010010202002  1216.564706  264.470588    0.000000     0.0   0.000000   
4       010010203001   334.094891    0.000000    0.000000     0.0   0.000000   
...              ...          ...         ...         ...     ...        ...   
219118  721537506011     0.000000    0.000000    0.000000     0.0   0.000000   
219119  721537506012     0.000000    0.000000    0.000000     0.0   0.000000   
219120  721537506013    29.582090   14.791045    0.000000     0.0   0.000000   
219121  721537506021   129.262295    0.000000    0.000000     0.0  38.778689   
219122  721537506022     0.000000   48.000000   48.000000    48.0   0.000000   

            hour_6      hour_7       hour_8       hour_9     hour_10  \
0         0.000000    0.000000     0.000000     0.000000    0.000000   
1         0.000000    0.000000     0.000000     0.000000    0.000000   
2         0.000000    0.000000     0.000000     0.000000  318.095238   
3       185.129412    0.000000  2023.200000  1256.235294  555.388235   
4         0.000000    0.000000   242.978102   415.087591  172.109489   
...            ...         ...          ...          ...         ...   
219118   12.095890    0.000000     0.000000    48.383562    0.000000   
219119   51.700820  103.401639    31.020492    10.340164   51.700820   
219120   22.186567   51.768657    22.186567    51.768657   59.164179   
219121   38.778689   77.557377   155.114754   155.114754  180.967213   
219122   24.000000   84.000000    84.000000    60.000000    0.000000   

           hour_11     hour_12     hour_13     hour_14     hour_15  \
0         0.000000  514.590164  275.245902  167.540984  227.377049   
1       207.358209  405.291045  405.291045  197.932836  197.932836   
2         0.000000  377.738095  318.095238    0.000000  357.857143   
3       806.635294  555.388235  277.694118  687.623529  793.411765   
4       587.197080  982.036496  799.802920  404.963504  627.693431   
...            ...         ...         ...         ...         ...   
219118   72.575342   36.287671   60.479452   36.287671   24.191781   
219119   62.040984   82.721311   72.381148   82.721311   51.700820   
219120   36.977612   29.582090   59.164179   36.977612   29.582090   
219121  168.040984  103.409836  116.336066  129.262295  116.336066   
219122  108.000000  108.000000   96.000000   36.000000  132.000000   

           hour_16     hour_17     hour_18      hour_19     hour_20  \
0         0.000000  179.508197  622.295082   263.278689  167.540984   
1       197.932836  386.440299  593.798507   782.305970  791.731343   
2         0.000000  437.380952  735.595238     0.000000    0.000000   
3       502.494118  952.094118  991.764706  1745.505882  436.376471   
4       809.927007  587.197080  647.941606   769.430657  526.452555   
...            ...         ...         ...          ...         ...   
219118   36.287671   60.479452   36.287671    12.095890    0.000000   
219119   20.680328   93.061475   72.381148    62.040984  134.422131   
219120   59.164179   73.955224   29.582090    14.791045   88.746269   
219121   90.483607   12.926230   25.852459    38.778689  103.409836   
219122  180.000000  168.000000   48.000000    12.000000   24.000000   

           hour_21     hour_22     hour_23     hour_24     hour_25  \
0       239.344262  227.377049  263.278689    0.000000    0.000000   
1       612.649254  612.649254  593.798507  367.589552  179.082090   
2       457.261905    0.000000  437.380952  477.142857    0.000000   
3         0.000000  185.129412  555.388235  290.917647  290.917647   
4       607.445255  809.927007  820.051095  860.547445  485.956204   
...            ...         ...         ...         ...         ...   
219118 

In [154]:
print("Restored actuall CBG hourly visitor (billion):")
print(restored_actual_CBG_hourly_visitor_df.iloc[:, 1:hour_cnt + 1].sum().sum()/ 1000000000)

Restored actuall CBG hourly visitor (billion):
51.028323628246746


In [155]:
restored_actual_target_hourly_visitor_df = pd.DataFrame()
restored_actual_target_hourly_visitor_df['stop'] = restored_actual_CBG_hourly_visitor_df.iloc[:, 1:hour_cnt + 1].sum(axis=0) 
print("Restored actuall target hourly visitor (billion):")
print(restored_actual_target_hourly_visitor_df['stop'].sum() / 1000000000)
restored_actual_target_hourly_visitor_df

Restored actuall target hourly visitor (billion):
51.028323628246746


stop
hour_1    3.619705e+07
hour_2    2.752242e+07
hour_3    2.458299e+07
hour_4    2.534076e+07
hour_5    3.077194e+07
...                ...
hour_716  9.390337e+07
hour_717  8.131823e+07
hour_718  6.743136e+07
hour_719  5.329988e+07
hour_720  3.596122e+07

[720 rows x 1 columns]

In [156]:
# save the restored actual visitor count
restored_actual_CBG_hourly_visitor_df.to_csv(restored_CBG_hourly_vistor_count_fname, index=False)
restored_actual_target_hourly_visitor_df.to_csv(restored_target_hourly_vistor_count_fname, index=False)

## IPF

### Load data

In [157]:
restored_actual_target_hourly_visitor_df = pd.read_csv(restored_target_hourly_vistor_count_fname, dtype={'CBG': str})
restored_actual_target_hourly_visitor_df

stop
0    3.619705e+07
1    2.752242e+07
2    2.458299e+07
3    2.534076e+07
4    3.077194e+07
..            ...
715  9.390337e+07
716  8.131823e+07
717  6.743136e+07
718  5.329988e+07
719  3.596122e+07

[720 rows x 1 columns]

In [160]:
print("Restored target total visitor count (billion):")
print(restored_actual_target_hourly_visitor_df['stop'].sum() / 1000000000) 

Restored target total visitor count (billion):
51.028323628246746


Load the CBG device distribution and compute the CBG aggregated restored visitors.

How to convert the CBG device distribution to stop distribution?

- option 1: convert the CBG device distribution to people distribution using device sampling rate, then using the people distribution as the stop distribution.
- option 2: compute the stop_per_device for each CBG, and then the stop for each home CBGZ, then aggregate the home CBG stops.

In [ ]:
import numpy as np
from ipfn import ipfn

restored_month_hour_outgoer_arr

aggregates = [restored_origin_arr, restored_month_hour_outgoer_arr]
dimensions = [[0], [1]]

seed_arr = np.ones((len(restored_origin_arr), len(restored_month_hour_outgoer_arr)))

IPF = ipfn.ipfn(seed_arr, aggregates, dimensions, convergence_rate=1e-3)
m = IPF.iteration()
print(m)